# install Package

In [15]:
!pip install transformers
!pip install --upgrade pip
!pip install --upgrade jedi
!pip install dask==2022.12.1
!pip install packaging>=21.3
!pip install protobuf>=3.20
!pip install rasa
!pip install nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.5 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2022.10.2
    Uninstalling dask-2022.10.2:
      Successfully uninstalled dask-2022.10.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rasa 3.6.0 requires dask==2022.10.2; python_version >= "3.8" and python_version < "3.11", but you have dask 2022.12.1 which is incompatible.
/bin/bash: =21.3: No such file or directory
/bin/bash: =3.20: No such file or directory
  Using cached dask-2022.10.2-py3-none-any.whl (1.1 MB)
  Attempting uninstall: dask
    Found existing installation: dask 2022.12.1
    Uninstalling dask-2022.12.1:
      Successfully uninstalled dask-2022.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the follow

# Libraries

In [1]:
import pandas as pd
import numpy as np
import transformers
import json

# Connect to Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
main_path = "/content/drive/MyDrive/NLP/CA6"
%cd {main_path}

/content/drive/MyDrive/NLP/CA6


# Functions

In [17]:
def generate_nlu_sub_intent(dataframe, file_path, operation_mode):
    with open(file_path, "w", encoding="utf-8") as output_file:
        output_file.write('version: "3.1"\n\n')
        output_file.write("nlu:\n")
        for index, record in dataframe.iterrows():
            output_file.write(f"\n- intent: faq/Q{record[0]}\n")
            output_file.write("  examples: |\n")
            if operation_mode == "train":
                for i in range(15):
                    output_file.write(f'    - "{record[7+i]}"\n')
            else:
                for i in range(5):
                    output_file.write(f'    - "{record[7+i]}"\n')

In [18]:
def generate_domain_sub_intent(dataframe, file_path):
    with open(file_path, "w", encoding="utf-8") as output_file:
        output_file.write('version: "3.1"\n\n')
        output_file.write("intents:\n")
        output_file.write("    - faq\n\n")
        output_file.write("responses:\n")
        for index, record in dataframe.iterrows():
            output_file.write(f"    utter_faq/Q{record[0]}:\n")
            output_file.write(f'    - text: "{record[1]}"\n')
        output_file.write("\nactions:\n")
        output_file.write("    - utter_faq\n")

In [19]:
def generate_rules_sub_intent(dataframe, file_path):
    with open(file_path, "w", encoding="utf-8") as output_file:
        output_file.write('version: "3.1"\n\n')


In [20]:
def generate_stories_sub_intent(dataframe, file_path):
    with open(file_path, "w", encoding="utf-8") as output_file:
        output_file.write('version: "3.1"\n\n')

In [21]:
def generate_classification_metrics(report_file_path):
    with open(report_file_path, "r") as report_file:
        evaluation_report = json.load(report_file)

    intent_metrics = evaluation_report["weighted avg"]
    accuracy = evaluation_report["accuracy"]
    metrics = {
        "Accuracy": accuracy,
        "Precision": intent_metrics["precision"],
        "Recall": intent_metrics["recall"],
        "F1 Score": intent_metrics["f1-score"]
    }

    return metrics

In [22]:
def generate_config_sub_intent(num_epochs, model_weights_name, config_file_path):
    config_content = f"""
language: fa
pipeline:
  - name: WhitespaceTokenizer
  - name: LanguageModelFeaturizer
    model_name: "bert"
    model_weights: "{model_weights_name}"
  - name: ResponseSelector
    epochs: {num_epochs}
    retrieval_intent: faq
    constrain_similarities: true
    cache_dir: null
policies:
  - name: MemoizationPolicy
  - name: TEDPolicy
    max_history: 5
    epochs: {num_epochs}
    constrain_similarities: true
  - name: RulePolicy
"""

    with open(config_file_path, 'w') as file:
        file.write(config_content)

In [23]:
def generate_stories_sub_intent(dataframe, file_path):
    with open(file_path, "w", encoding="utf-8") as output_file:
        output_file.write('version: "3.1"\n\n')

In [24]:
def generate_nlu_multi_intent(dataframe, file_path, mode):
     with open(file_path, "w", encoding="utf-8") as file:
        file.write('version: "3.1"\n\n')
        file.write("nlu:\n")
        for index, record in dataframe.iterrows():
            file.write(f"\n- intent: Q{record[0]}\n")
            file.write("  examples: |\n")
            if mode == "train":
                for i in range(15):
                    file.write(f'    - "{record[7+i]}"\n')
            else:
                for i in range(5):
                    file.write(f'    - "{record[7+i]}"\n')

In [25]:
def generate_domain_multi_intent(dataframe, file_path):
    with open(file_path, "w", encoding="utf-8") as file:
        file.write('version: "3.1"\n\n')
        file.write("intents:\n")

        for index, record in dataframe.iterrows():
            file.write(f"    - q{record[0]}\n")

        file.write("\nresponses:\n")
        for index, record in dataframe.iterrows():
            file.write(f"    utter_Q{record[0]}:\n")
            file.write(f'    - text: "{record[1]}"\n')

In [26]:
def generate_rules_multi_intent(dataframe, file_path):
    with open(file_path, "w", encoding="utf-8") as file:
        file.write('version: "3.1"\n\n')
        file.write("rules:\n")

        for index, record in dataframe.iterrows():
            file.write(f"    - rule: response to Q{record[0]}\n")
            file.write("      steps:\n")
            file.write(f"      - intent: Q{record[0]}\n")
            file.write(f'      - action: utter_Q{record[0]}\n')

In [27]:
def generate_stories_multi_intent(dataframe, file_path):
    with open(file_path, "w", encoding="utf-8") as file:
        file.write('version: "3.1"\n\n')

In [28]:
def generate_config_multi_intent(num_epochs, model_weights_name, config_file_path):
    config_content = f"""
language: fa
pipeline:
  - name: WhitespaceTokenizer
  - name: LanguageModelFeaturizer
    model_name: "bert"
    model_weights: "{model_weights_name}"
  - name: ResponseSelector
    epochs: {num_epochs}
    constrain_similarities: true
    cache_dir: null
  - name: DIETClassifier
    epochs: {num_epochs}
    constrain_similarities: true
    cache_dir: null
policies:
  - name: MemoizationPolicy
  - name: TEDPolicy
    max_history: 5
    constrain_similarities: true
  - name: RulePolicy
"""

    with open(config_file_path, 'w') as file:
        file.write(config_content)


# Sub-intent Scenario

In [20]:
# !mkdir -p sub_intent_folder
%cd ./sub_intent_folder


# create a folder for test
# ! mkdir -p test_folder

/content/drive/MyDrive/NLP/CA6/sub_intent_folder


Initializ a new Rasa project without interactive prompts

In [21]:
%%shell
rasa init --no-prompt

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

## Load Data

In [22]:
def read_file(path):
  df = pd.read_excel(path)
  return df

In [23]:
train_df = read_file(main_path + "/Data/MCI_Internet_TrainData.xlsx")
test_df  = read_file(main_path + "/Data/MCI_Internet_TestData.xlsx")

### Write data in folders

In [24]:
generate_nlu_sub_intent(train_df, f'{main_path}/sub_intent_folder/data/nlu.yml', "train")
generate_rules_sub_intent(train_df, f'{main_path}/sub_intent_folder/data/rules.yml')
generate_stories_sub_intent(train_df, f'{main_path}/sub_intent_folder/data/stories.yml')
generate_domain_sub_intent(train_df, f'{main_path}/sub_intent_folder/domain.yml')
generate_nlu_sub_intent(test_df,f'{main_path}/sub_intent_folder/test_folder/nlu_test.yml', "test" )

# /content/drive/MyDrive/NLP/CA6/sub_intent_folder/sub_intent_folder/test_folder

## ParseBert with 50 epochs

### Configuration

In [41]:
generate_config_sub_intent(num_epochs=50,model_weights_name="HooshvareLab/bert-base-parsbert-uncased", config_file_path = './config.yml')

### Train

In [42]:
%%shell
rasa train

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Test

In [43]:
%%shell
rasa test nlu --config config.yml \
              --nlu test_folder/nlu_test.yml\
              --out test_result/config

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Result

In [44]:
report_file_path =  "test_result/config/response_selection_report.json"
generate_classification_metrics(report_file_path)

{'Accuracy': 0.624,
 'Precision': 0.6531746031746033,
 'Recall': 0.624,
 'F1 Score': 0.5914615188732836}

## LabSE with 50 epochs

### Configuration

In [45]:
generate_config_sub_intent(num_epochs=50,model_weights_name="rasa/LaBSE", config_file_path = './config.yml')

### Train

In [46]:
%%shell
rasa train

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Test

In [47]:
%%shell
rasa test nlu --config config.yml \
              --nlu test_folder/nlu_test.yml\
              --out test_result/config

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Result

In [48]:
report_file_path =  "test_result/config/response_selection_report.json"
generate_classification_metrics(report_file_path)

{'Accuracy': 0.908,
 'Precision': 0.9255238095238095,
 'Recall': 0.908,
 'F1 Score': 0.9070089910089909}

LabSE is better so do it for 100 and 200 epochs

## LabSE with 100 epochs

### Configuration

In [49]:
generate_config_sub_intent(num_epochs=100, model_weights_name="rasa/LaBSE", config_file_path = './config.yml')

### Train

In [50]:
%%shell
rasa train

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Test

In [51]:
%%shell
rasa test nlu --config config.yml \
              --nlu test_folder/nlu_test.yml\
              --out test_result/config

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Result

In [52]:
report_file_path =  "test_result/config/response_selection_report.json"
generate_classification_metrics(report_file_path)

{'Accuracy': 0.912,
 'Precision': 0.9222857142857144,
 'Recall': 0.912,
 'F1 Score': 0.9089584859584859}

## LabSE with 200 epochs

### Configuration

In [53]:
generate_config_sub_intent(num_epochs=200, model_weights_name="rasa/LaBSE", config_file_path = './config.yml')

### Train

In [54]:
%%shell
rasa train

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Test

In [55]:
%%shell
rasa test nlu --config config.yml \
              --nlu test_folder/nlu_test.yml\
              --out test_result/config

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Result

In [56]:
report_file_path =  "test_result/config/response_selection_report.json"
generate_classification_metrics(report_file_path)

{'Accuracy': 0.916,
 'Precision': 0.923,
 'Recall': 0.916,
 'F1 Score': 0.9143636363636364}

# Multi-intent Scenario

In [30]:
# !mkdir -p multi_intent_folder
%cd ./multi_intent_folder


# create a folder for test
# ! mkdir -p test_folder

/content/drive/MyDrive/NLP/CA6/multi_intent_folder


In [35]:
%pwd

'/content/drive/MyDrive/NLP/CA6/multi_intent_folder'

Initializ a new Rasa project without interactive prompts

In [36]:
%%shell
rasa init --no-prompt

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

## Load Data

In [39]:
def read_file(path):
  df = pd.read_excel(path)
  return df

In [40]:
print(main_path)

/content/drive/MyDrive/NLP/CA6


In [41]:
train_df = read_file(main_path + "/Data/MCI_Internet_TrainData.xlsx")
test_df  = read_file(main_path + "/Data/MCI_Internet_TestData.xlsx")

### Write data in folders

In [42]:
generate_nlu_multi_intent(train_df, f'{main_path}/multi_intent_folder/data/nlu.yml', "train")
generate_rules_multi_intent(train_df, f'{main_path}/multi_intent_folder/data/rules.yml')
generate_stories_multi_intent(train_df, f'{main_path}/multi_intent_folder/data/stories.yml')
generate_domain_multi_intent(train_df, f'{main_path}/multi_intent_folder/domain.yml')
generate_nlu_multi_intent(test_df,f'{main_path}/multi_intent_folder/test_folder/nlu_test.yml', "test")

## ParseBert with 50 epochs

### Configuration

In [43]:
generate_config_multi_intent(num_epochs=50, model_weights_name="HooshvareLab/bert-base-parsbert-uncased", config_file_path = './config.yml')

### Train

In [44]:
%%shell
rasa train

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Test

In [45]:
%%shell
rasa test nlu --config config.yml \
              --nlu test_folder/nlu_test.yml\
              --out test_result/config

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Result

In [47]:
report_file_path =  "test_result/config/intent_report.json"
generate_classification_metrics(report_file_path)

{'Accuracy': 0.776,
 'Precision': 0.7907936507936507,
 'Recall': 0.776,
 'F1 Score': 0.7584115884115884}

## LabSE with 50 epochs

### Configuration

In [48]:
generate_config_multi_intent(num_epochs=50, model_weights_name="rasa/LaBSE", config_file_path = './config.yml')

### Train

In [49]:
%%shell
rasa train

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Test

In [50]:
%%shell
rasa test nlu --config config.yml \
              --nlu test_folder/nlu_test.yml\
              --out test_result/config

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Result

In [51]:
report_file_path =  "test_result/config/intent_report.json"
generate_classification_metrics(report_file_path)

{'Accuracy': 0.908,
 'Precision': 0.9187857142857143,
 'Recall': 0.908,
 'F1 Score': 0.9046212676212675}

LabSE is better so do it for 100 and 200 epochs

## LabSE with 100 epochs

### Configuration

In [52]:
generate_config_multi_intent(num_epochs=100, model_weights_name="rasa/LaBSE", config_file_path = './config.yml')

### Train

In [53]:
%%shell
rasa train

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Test

In [54]:
%%shell
rasa test nlu --config config.yml \
              --nlu test_folder/nlu_test.yml\
              --out test_result/config

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Result

In [57]:
report_file_path =  "test_result/config/intent_report.json"
generate_classification_metrics(report_file_path)

{'Accuracy': 0.904,
 'Precision': 0.9246190476190476,
 'Recall': 0.904,
 'F1 Score': 0.901968253968254}

## LabSE with 200 epochs

### Configuration

In [58]:
generate_config_multi_intent(num_epochs=200, model_weights_name="rasa/LaBSE", config_file_path = './config.yml')

### Train

In [59]:
%%shell
rasa train

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Test

In [60]:
%%shell
rasa test nlu --config config.yml \
              --nlu test_folder/nlu_test.yml\
              --out test_result/config

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1042: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP

### Result

In [61]:
report_file_path =  "test_result/config/intent_report.json"
generate_classification_metrics(report_file_path)

{'Accuracy': 0.928,
 'Precision': 0.9376190476190476,
 'Recall': 0.928,
 'F1 Score': 0.9251659451659451}